# Import modules

In [2]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm, block_diag
from scipy.constants import h,k, hbar
from math import pi
from qutip import *
import sympy
import functools, operator
from collections import OrderedDict
import itertools

from transmonCPW import Transmon

Using matplotlib backend: TkAgg


In [3]:
N=4
b =destroy(N)
b.dag()*b
# num(N)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 3.]]

# Class/function

In [4]:
def hamiltonian_transmon_CSFQ_bus(g1, g2, g12, w1, w2, wc, alpha1, alpha2, N):
    """
    Full Hamiltonian of two transmons and a coupler.
    Note: This produces negative ZZ as experimentally shown in Li paper.
    """
    b = destroy(N)
    I = qeye(N)
    
    # Eq. A17
    H1 = w1*tensor(b.dag()*b, I,I) + alpha1/2.0*tensor(b.dag()*b.dag()*b*b, I, I)
    Hc = wc*tensor(I, num(N), I)
    H2 = w2*tensor(I, I, b.dag()*b) + alpha2/2.0*tensor(I, I, b.dag()*b.dag()*b*b)
    H1c = g1*(tensor(b.dag(), b, I) + tensor(b, b.dag(), I))
    H2c = g2*(tensor(I, b.dag(), b) + tensor(I, b, b.dag()))
    H12 = g12*(tensor(b.dag(), I, b) + tensor(b, I, b.dag()))
    
    H_eff = H1 + H2 + Hc + H1c + H2c + H12
    
    return H_eff

def pad_zero(s):
    if len(s) == 3:
        return s
    else:
        return '0'*(3-len(s)) + f'{s}'

def get_bare_ket_rep_from_eigenstate(eigst):
    N = int(len(np.array(eigst)) ** (1/3))
    max_index = np.argmax(np.abs(eigst))
    base_3_rep = np.base_repr(max_index, base=N)
    base_3_rep_zero_pad = pad_zero(base_3_rep)
    
    return f'|{base_3_rep_zero_pad}>'

def show_eigenstate_in_order(eigenstate):
    """
    eigenstate: e2 from "e1, e2 = H.eigenstates()"" in qubitp
    """
    
    for eigst in eigenstate:
        base_3_rep = get_bare_ket_rep_from_eigenstate(eigst)
        print(base_3_rep)
                

# Transmon params

In [5]:
def print_result(q1):
    l = [q1.total_cap/1e-15, q1.anharmonicity/1e6, q1.Ej_to_Ec_ratio, q1.f01/1e9]
    print(f'Ej = {q1.Ej/h/1e9:.3g} GHz')
    print(f'Ec = {q1.Ec/h/1e9:.3g} GHz')
    print(f'Total Cap = {q1.total_cap/1e-15:.3g} fF')
    print(f'Anharm. = {q1.anharmonicity/1e6:.3g} MHz')
    print(f'Ej/Ec = {q1.Ej_to_Ec_ratio:.3g}')
    print(f'f01 = {q1.f01/1e9:.3g} GHz')
    print()
    
q1 = Transmon.in_GHz(13, 0.28)
# q2 = Transmon.in_GHz(22, 0.3)


print_result(q1)
# print_result(q2)

q2 = Transmon.in_GHz(17.5, 0.35)
q2.charge_dispersion/1e6
# q2.anharmonicity

Ej = 13 GHz
Ec = 0.28 GHz
Total Cap = 69.2 fF
Anharm. = -324 MHz
Ej/Ec = 46.4
f01 = 5.1 GHz



0.013929796595573425

# CSFQ /  Bus / Asymmetric Transmon

In [13]:
# parameters for IBM qubit
w1, w2, wc =  5.9, 4.9, 5.3 # in GHz
alpha1, alpha2 =  -0.35, 0.5# in GHz

(g1, g2) = 0.133,0.128
g12 = 2/wc*g1*g2
N = 3
w1_list = np.linspace(5.7, 6.7, 51)
energy_levels = np.zeros((N**3, w1_list.size), dtype=complex)
ZZ_list = np.zeros(w1_list.size)

for idx, w1 in enumerate(w1_list):
    H_eff = hamiltonian_transmon_CSFQ_bus(g1, g2, g12, w1, w2, wc, alpha1, alpha2, N)
    eigval, eigstate = H_eff.eigenstates()
    energy_levels[:,idx] =  eigval
    ZZ_list[idx] = eigval[7] - eigval[1] - eigval[3] + eigval[0]   # in GHz

how_many_levels = 10
fig,(ax1, ax2) = plt.subplots(1,2, figsize=(14,8))
for i in range(1, how_many_levels + 1):
    ax1.plot(w1_list, energy_levels[i,:],linewidth=2)
    ax1.annotate(get_bare_ket_rep_from_eigenstate(eigstate[i]), fontsize=14,
            xy=(w1_list[-1]+0.02, energy_levels[i,-1]), xycoords='data',
            xytext=(w1_list[-1]+0.1, energy_levels[i,-1]), textcoords='data',
            arrowprops=dict(arrowstyle="-",
                            connectionstyle="arc3"),
            )
ax1.set_xlim(right=(w1_list[-1]+0.3))
ax1.set_xlabel('Transmon Freq (GHz)', fontsize=18)
ax1.set_ylabel('Eneryg Level (GHz)', fontsize=18)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
# ax1.grid(axis='x')

ax2.plot(w1_list, ZZ_list*1e3)
ax2.set_xlabel('Transmon Freq (GHz)', fontsize=18)
ax2.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax2.tick_params(axis='x', labelsize=18)
ax2.tick_params(axis='y', labelsize=18)
ax2.grid('on')


/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/usr/lib/python3/dist-packages/matplotlib/text.py:1761: ComplexWarning: Casting complex values to real discards the imaginary part
  y = float(self.convert_yunits(y))
/usr/lib/python3/dist-packages/matplotlib/text.py:814: ComplexWarning: Casting complex values to real discards the imaginary part
  y = float(self.convert_yunits(self._y))
/usr/lib/python3/dist-packages/matplotlib/text.py:676: ComplexWarning: Casting complex values to real discards the imaginary part
  posy = float(textobj.convert_yunits(textobj._y))


In [7]:
fig, ax = plt.subplots(1,1, figsize=(8,8))
# for i in range(len(eigstate)):
for i in range(10):
    ax.plot( np.array(eigstate[i]) + i*1.2,'-o')
ax.set_xlabel('Index', fontsize=18)
# ax.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

NameError: name 'eigstate' is not defined

In [285]:
show_eigenstate_in_order(eigstate)

|000>
|001>
|010>
|100>
|011>
|002>
|020>
|101>
|110>
|200>
|021>
|012>
|111>
|102>
|120>
|201>
|210>
|022>
|121>
|112>
|211>
|202>
|220>
|122>
|221>
|212>
|222>
